# Getting Started with Adaptive Liquidity Provision Engine

This notebook demonstrates the basic usage of the market making framework.

## Topics Covered:
1. Market simulation basics
2. Order book dynamics
3. Running baseline strategies
4. Evaluating performance

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.simulation.market_simulator import MarketSimulator, SimulationConfig
from src.simulation.order_book import Order, OrderType, OrderSide
from src.environments.market_making_env import MarketMakingEnv, MarketMakingConfig
from src.agents.baseline_agents import StaticSpreadAgent, AvellanedaStoikovAgent
from src.evaluation.backtester import Backtester

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful!")

## 1. Market Simulation Basics

Let's start by creating a simple market simulator and observing its behavior.

In [ ]:
# Create simulator
config = SimulationConfig(
    initial_midprice=100.0,
    initial_spread=0.02,
    tick_size=0.01,
    seed=42
)

sim = MarketSimulator(config)

# Run simulation
print("Running simulation for 100 seconds...")
stats = sim.run_simulation(duration=100.0, time_step=1.0)

print("\nSimulation Statistics:")
for key, value in stats.items():
    if key != 'final_state':
        print(f"  {key}: {value}")

### Visualize Market State Evolution

In [ ]:
# Extract time series
states = sim.get_state_history()
timestamps = [s.timestamp for s in states]
midprices = [s.midprice for s in states]
spreads = [s.spread for s in states]
imbalances = [s.imbalance for s in states]
volatilities = [s.volatility for s in states]

# Plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].plot(timestamps, midprices, linewidth=2, color='#2E86AB')
axes[0, 0].set_title('Midprice Evolution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].grid(alpha=0.3)

axes[0, 1].plot(timestamps, spreads, linewidth=2, color='#A23B72')
axes[0, 1].set_title('Bid-Ask Spread', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Spread ($)')
axes[0, 1].grid(alpha=0.3)

axes[1, 0].plot(timestamps, imbalances, linewidth=2, color='#06D6A0')
axes[1, 0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Order Book Imbalance', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Imbalance')
axes[1, 0].grid(alpha=0.3)

axes[1, 1].plot(timestamps, volatilities, linewidth=2, color='#F18F01')
axes[1, 1].set_title('Volatility Regime', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Volatility')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Average Midprice: ${np.mean(midprices):.2f}")
print(f"📊 Average Spread: ${np.mean(spreads):.4f}")
print(f"📊 Average Volatility: {np.mean(volatilities):.4f}")

## 2. Order Book Visualization

Let's examine the current state of the order book.

In [ ]:
# Get order book snapshot
book_state = sim.lob.get_state_snapshot()

print("Order Book State:")
print(f"  Best Bid: ${book_state['best_bid']:.2f}")
print(f"  Best Ask: ${book_state['best_ask']:.2f}")
print(f"  Midprice: ${book_state['midprice']:.2f}")
print(f"  Spread: ${book_state['spread']:.4f}")
print(f"  Imbalance: {book_state['imbalance']:.3f}")
print(f"  Total Volume: {book_state['total_volume']:,}")
print(f"  Total Trades: {book_state['total_trades']:,}")

# Visualize depth
depth = book_state['depth']

fig, ax = plt.subplots(figsize=(12, 6))

if depth['bids']:
    bid_prices, bid_sizes = zip(*depth['bids'])
    ax.barh(bid_prices, bid_sizes, color='#06D6A0', alpha=0.7, label='Bids')

if depth['asks']:
    ask_prices, ask_sizes = zip(*depth['asks'])
    ax.barh(ask_prices, [-s for s in ask_sizes], color='#EF476F', alpha=0.7, label='Asks')

ax.axhline(y=book_state['midprice'], color='black', linestyle='--', linewidth=2, label='Midprice')
ax.set_xlabel('Size (negative = ask side)', fontsize=12)
ax.set_ylabel('Price ($)', fontsize=12)
ax.set_title('Order Book Depth (L2)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Running Baseline Strategies

Now let's test some baseline market making strategies.

In [ ]:
# Create environment
env_config = MarketMakingConfig(
    episode_duration=100.0,
    time_step=1.0
)
env = MarketMakingEnv(env_config)

# Create backtester
backtester = Backtester(env, verbose=True)

# Test agents
agents = [
    (StaticSpreadAgent(spread_ticks=2, quote_size=100), "Static Spread"),
    (AvellanedaStoikovAgent(quote_size=100), "Avellaneda-Stoikov")
]

print("\n" + "="*80)
print("TESTING BASELINE STRATEGIES")
print("="*80)

results = backtester.compare_agents(agents, num_episodes=5)

### Visualize Results

In [ ]:
# Plot P&L comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for name, result in results.items():
    axes[0].plot(result.timestamps, result.pnl_series, label=name, linewidth=2)
    axes[1].plot(result.timestamps, result.inventory_series, label=name, linewidth=2)

axes[0].set_title('P&L Comparison', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('P&L ($)')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

axes[1].set_title('Inventory Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Inventory')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

### Performance Metrics Table

In [ ]:
# Create comparison dataframe
metrics_data = []
for name, result in results.items():
    metrics_data.append({
        'Agent': name,
        'Total P&L': f"${result.metrics.total_pnl:.2f}",
        'Sharpe Ratio': f"{result.metrics.sharpe_ratio:.3f}",
        'Max Drawdown': f"{result.metrics.max_drawdown:.2%}",
        'Win Rate': f"{result.metrics.win_rate:.2%}",
        'Avg |Inventory|': f"{result.metrics.mean_abs_inventory:.1f}",
        'Profit Factor': f"{result.metrics.profit_factor:.2f}"
    })

df_metrics = pd.DataFrame(metrics_data)
display(df_metrics)

## 4. Single Episode Deep Dive

Let's examine a single episode in detail.

In [ ]:
# Run single episode
agent = AvellanedaStoikovAgent()
episode_data = backtester.run_episode(agent)

# Extract data
infos = episode_data['infos']
rewards = episode_data['rewards']
actions = episode_data['actions']

# Plot detailed view
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Rewards
axes[0].plot(rewards, linewidth=2, color='#2E86AB')
axes[0].set_title('Step Rewards', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Reward')
axes[0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0].grid(alpha=0.3)

# P&L
pnls = [info['pnl'] for info in infos]
axes[1].plot(pnls, linewidth=2, color='#06D6A0')
axes[1].set_title('Cumulative P&L', fontsize=14, fontweight='bold')
axes[1].set_ylabel('P&L ($)')
axes[1].grid(alpha=0.3)

# Inventory
inventories = [info['inventory'] for info in infos]
axes[2].plot(inventories, linewidth=2, color='#A23B72')
axes[2].fill_between(range(len(inventories)), inventories, alpha=0.3)
axes[2].set_title('Inventory Position', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Inventory')
axes[2].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Episode Summary:")
print(f"   Total Reward: {sum(rewards):.2f}")
print(f"   Final P&L: ${episode_data['final_pnl']:.2f}")
print(f"   Final Inventory: {episode_data['final_inventory']}")
print(f"   Steps: {episode_data['num_steps']}")

## Next Steps

- **Notebook 2**: Training RL agents with PPO
- **Notebook 3**: Advanced market impact analysis
- **Notebook 4**: Portfolio of market makers

---

For more information, see the [documentation](../docs/).